In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from autogluon.tabular import TabularPredictor

# ----------------------
# 0. 설정
# ----------------------
BASE_PATH = "open_track1/"
PATH_TRAIN = os.path.join(BASE_PATH, "train.csv")
PATH_TEST = os.path.join(BASE_PATH, "test.csv")
PATH_MATCH_INFO = os.path.join(BASE_PATH, "match_info.csv")
PATH_SAMPLE_SUB = os.path.join(BASE_PATH, "sample_submission.csv")

K = 20   # 마지막 K 이벤트 사용 (20~32 사이 선택)

# ----------------------
# 1. 데이터 로드
# ----------------------
train = pd.read_csv(PATH_TRAIN)
test_index = pd.read_csv(PATH_TEST)
match_info = pd.read_csv(PATH_MATCH_INFO)
sample_sub = pd.read_csv(PATH_SAMPLE_SUB)

test_events_list = []
for _, row in test_index.iterrows():
    # path가 "./test/..." 형식이므로 BASE_PATH와 결합
    test_path = os.path.join(BASE_PATH, row["path"].lstrip("./"))
    df_ep = pd.read_csv(test_path)
    test_events_list.append(df_ep)

test_events = pd.concat(test_events_list, ignore_index=True)

train["is_train"] = 1
test_events["is_train"] = 0

events = pd.concat([train, test_events], ignore_index=True)

# ----------------------
# 2. 기본 정렬 + episode 내 인덱스
# ----------------------
events = events.sort_values(["game_episode", "time_seconds", "action_id"]).reset_index(drop=True)

events["event_idx"] = events.groupby("game_episode").cumcount()
events["n_events"] = events.groupby("game_episode")["event_idx"].transform("max") + 1
events["ep_idx_norm"] = events["event_idx"] / (events["n_events"] - 1).clip(lower=1)

# ----------------------
# 3. 시간/공간 feature
# ----------------------
# Δt
events["prev_time"] = events.groupby("game_episode")["time_seconds"].shift(1)
events["dt"] = events["time_seconds"] - events["prev_time"]
events["dt"] = events["dt"].fillna(0.0)

# 이동량/거리
events["dx"] = events["end_x"] - events["start_x"]
events["dy"] = events["end_y"] - events["start_y"]
events["dist"] = np.sqrt(events["dx"]**2 + events["dy"]**2)

# 속도 (dt=0 보호)
events["speed"] = events["dist"] / events["dt"].replace(0, 1e-3)

# zone / lane (필요시 범위 조정)
events["x_zone"] = (events["start_x"] / (105/7)).astype(int).clip(0, 6)
events["lane"] = pd.cut(
    events["start_y"],
    bins=[0, 68/3, 2*68/3, 68],
    labels=[0, 1, 2],
    include_lowest=True
).astype(int)

# ----------------------
# 3-1. 코너 관련 피처 (Corner Features)
# ----------------------
# 골대까지 각도 계산 (라디안 → 도)
# 골대 중앙: (105, 34)
events["angle_to_goal"] = np.arctan2(
    34 - events["start_y"],
    105 - events["start_x"]
) * 180 / np.pi

# 코너까지 최단 거리 계산
# 공격 방향 코너: (105, 0) 상단, (105, 68) 하단
events["dist_corner_top"] = np.sqrt((105 - events["start_x"])**2 + (0 - events["start_y"])**2)
events["dist_corner_bottom"] = np.sqrt((105 - events["start_x"])**2 + (68 - events["start_y"])**2)
events["dist_to_nearest_corner"] = events[["dist_corner_top", "dist_corner_bottom"]].min(axis=1)

# 코너 구역 플래그 (X > 100 이면서 Y < 5 또는 Y > 63)
events["is_corner_area"] = ((events["start_x"] > 100) & 
                            ((events["start_y"] < 5) | (events["start_y"] > 63))).astype(int)

# 인터랙션 피처 1: angle_to_goal × is_corner_area
# 코너 구역일 때만 각도의 영향이 강하게 나타남
events["angle_goal_x_corner"] = events["angle_to_goal"] * events["is_corner_area"]

# 인터랙션 피처 2: dist_to_nearest_corner × angle_to_goal
# 코너 거리와 각도의 조합
events["dist_corner_x_angle"] = events["dist_to_nearest_corner"] * events["angle_to_goal"]

# ----------------------
# 3-2. 새로운 핵심 피처 3개 (고득점용)
# ----------------------
# ① 사이드라인 압박도 (dist_to_sideline)
# 의도: 터치라인에 붙을수록 패스 각도가 제한되는 '공간적 제약'을 수치화합니다.
events["dist_to_sideline"] = events["start_y"].apply(lambda y: min(y, 68-y))

# ② 골대 중심과의 물리적 각도 (angle_to_goal_center)
# 의도: 단순히 원점 기준 각도가 아니라, 목적지인 골대를 기준으로 한 각도를 줍니다.
# (라디안 단위로 저장 - 기존 angle_to_goal은 도 단위)
events["angle_to_goal_center"] = np.arctan2(
    34 - events["start_y"], 
    105 - events["start_x"]
)

# ③ 시간 × 전진 위치 상호작용 (time_pos_interaction)
# 의도: "후반전 + 상대 진영"이라는 조건이 만났을 때의 특수 상황을 강조합니다.
events["time_pos_inter"] = events["ep_idx_norm"] * events["start_x"]

print("✓ 핵심 피처 3개 추가 완료 (dist_to_sideline, angle_to_goal_center, time_pos_inter)")

# ----------------------
# 4. 라벨 및 episode-level 메타 (train 전용)
# ----------------------
train_events = events[events["is_train"] == 1].copy()

last_events = (
    train_events
    .groupby("game_episode", as_index=False)
    .tail(1)
    .copy()
)

labels = last_events[["game_episode", "end_x", "end_y"]].rename(
    columns={"end_x": "target_x", "end_y": "target_y"}
)

# episode-level 메타 (마지막 이벤트 기준)
ep_meta = last_events[["game_episode", "game_id", "team_id", "is_home", "period_id", "time_seconds"]].copy()
ep_meta = ep_meta.rename(columns={"team_id": "final_team_id"})

# game_clock (분 단위, 0~90+)
ep_meta["game_clock_min"] = np.where(
    ep_meta["period_id"] == 1,
    ep_meta["time_seconds"] / 60.0,
    45.0 + ep_meta["time_seconds"] / 60.0
)

# ----------------------
# 5. 공격 팀 플래그 (final_team vs 상대)
# ----------------------
# final_team_id를 전체 events에 붙임
events = events.merge(
    ep_meta[["game_episode", "final_team_id"]],
    on="game_episode",
    how="left"
)

events["is_final_team"] = (events["team_id"] == events["final_team_id"]).astype(int)

# ----------------------
# 6. 입력용 events에서 마지막 이벤트 타깃 정보 가리기
# ----------------------
# is_last 플래그
events["last_idx"] = events.groupby("game_episode")["event_idx"].transform("max")
events["is_last"] = (events["event_idx"] == events["last_idx"]).astype(int)

# labels는 이미 뽑아놨으니, 입력쪽에서만 end_x, end_y, dx, dy, dist, speed 지움
mask_last = events["is_last"] == 1
for col in ["end_x", "end_y", "dx", "dy", "dist", "speed"]:
    events.loc[mask_last, col] = np.nan

# ----------------------
# 7. 카테고리 인코딩 (type_name, result_name, team_id 등)
# ----------------------
events["type_name"] = events["type_name"].fillna("__NA_TYPE__")
events["result_name"] = events["result_name"].fillna("__NA_RES__")

le_type = LabelEncoder()
le_res = LabelEncoder()

events["type_id"] = le_type.fit_transform(events["type_name"])
events["res_id"] = le_res.fit_transform(events["result_name"])

# team_id는 그대로 써도 되지만, 문자열이면 숫자로 매핑
if events["team_id"].dtype == "object":
    le_team = LabelEncoder()
    events["team_id_enc"] = le_team.fit_transform(events["team_id"])
else:
    events["team_id_enc"] = events["team_id"].astype(int)

# ----------------------
# 8. 마지막 K 이벤트만 사용 (lastK)
# ----------------------
# rev_idx: 0이 마지막 이벤트
events["rev_idx"] = events.groupby("game_episode")["event_idx"].transform(
    lambda s: s.max() - s
)

lastK = events[events["rev_idx"] < K].copy()

# pos_in_K: 0~(K-1), 앞쪽 패딩 고려해서 뒤에 실제 이벤트가 모이게
def assign_pos_in_K(df):
    df = df.sort_values("event_idx")  # 오래된 → 최근
    L = len(df)
    df = df.copy()
    df["pos_in_K"] = np.arange(K - L, K)
    return df

lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos_in_K)

# ----------------------
# 9. wide feature pivot
# ----------------------
# 사용할 이벤트 피처 선택
num_cols = [
    "start_x", "start_y",
    "end_x", "end_y",
    "dx", "dy", "dist", "speed",
    "dt",
    "ep_idx_norm",
    "x_zone", "lane",
    "is_final_team",
    # 코너 관련 피처
    "angle_to_goal",
    "dist_to_nearest_corner",
    "is_corner_area",
    "angle_goal_x_corner",  # 인터랙션: angle_to_goal × is_corner_area
    "dist_corner_x_angle",  # 인터랙션: dist_to_nearest_corner × angle_to_goal
    # 새로운 핵심 피처 3개
    "dist_to_sideline",      # 사이드라인 압박도
    "angle_to_goal_center",  # 골대 중심과의 물리적 각도
    "time_pos_inter",        # 시간 × 전진 위치 상호작용
]

cat_cols = [
    "type_id",
    "res_id",
    "team_id_enc",
    "is_home",
    "period_id",
    "is_last",
]

feature_cols = num_cols + cat_cols

wide = lastK[["game_episode", "pos_in_K"] + feature_cols].copy()

# 숫자형 pivot
wide_num = wide.pivot_table(
    index="game_episode",
    columns="pos_in_K",
    values=num_cols,
    aggfunc="first"
)

# 범주형 pivot
wide_cat = wide.pivot_table(
    index="game_episode",
    columns="pos_in_K",
    values=cat_cols,
    aggfunc="first"
)

# 컬럼 이름 평탄화
wide_num.columns = [f"{c}_{int(pos)}" for (c, pos) in wide_num.columns]
wide_cat.columns = [f"{c}_{int(pos)}" for (c, pos) in wide_cat.columns]

X = pd.concat([wide_num, wide_cat], axis=1).reset_index()  # game_episode 포함

# episode-level 메타 붙이기
X = X.merge(ep_meta[["game_episode", "game_id", "game_clock_min", "final_team_id", "is_home", "period_id"]],
            on="game_episode", how="left")

# train 라벨 붙이기
X = X.merge(labels, on="game_episode", how="left")  # test는 NaN

# ----------------------
# 10. train/test 분리
# ----------------------
train_mask = X["game_episode"].isin(labels["game_episode"])
X_train = X[train_mask].copy()
X_test = X[~train_mask].copy()

y_train_x = X_train["target_x"].astype(float)
y_train_y = X_train["target_y"].astype(float)

# 모델 입력에서 빼야 할 컬럼들
drop_cols = [
    "game_episode",
    "game_id",
    "target_x",
    "target_y",
]

X_train_feat = X_train.drop(columns=drop_cols)
X_test_feat = X_test.drop(columns=[c for c in drop_cols if c in X_test.columns])

# NaN 채우기 (LGBM은 NaN 다루긴 하지만, 깔끔하게)
X_train_feat = X_train_feat.fillna(0)
X_test_feat = X_test_feat.fillna(0)

# ----------------------
# 11. AutoGluon 학습
# ----------------------
# CatBoost와 LightGBM 사용 (30분)
hyperparameters = {
    'CAT': {},
    'GBM': {},  # LightGBM
}
time_limit = 1800  # 30분
presets = "good_quality"
print("=" * 50)
print("CatBoost + LightGBM 사용 (30분)")
print("=" * 50)

# X 좌표 예측 모델
print("\n" + "=" * 50)
print("X 좌표 모델 학습 시작...")
print("=" * 50)

train_data_x = X_train_feat.copy()
train_data_x["target_x"] = y_train_x

# 모델 저장 경로
model_suffix = "sideline_goalangle_timepos"
model_path_x = f"ag_models_x_{model_suffix}"
model_path_y = f"ag_models_y_{model_suffix}"

print(f"\n모델 저장 경로: {model_path_x}, {model_path_y}")
print(f"사용 모델: CatBoost + LightGBM")
print(f"사용 피처: dist_to_sideline, angle_to_goal_center, time_pos_inter")

predictor_x = TabularPredictor(
    label="target_x",
    problem_type="regression",
    eval_metric="rmse",
    path=model_path_x
).fit(
    train_data=train_data_x,
    time_limit=time_limit,
    presets=presets,
    hyperparameters=hyperparameters,  # 특정 모델만 선택
    verbosity=2
)

print("\n" + "=" * 50)
print("Y 좌표 모델 학습 시작...")
print("=" * 50)

train_data_y = X_train_feat.copy()
train_data_y["target_y"] = y_train_y

predictor_y = TabularPredictor(
    label="target_y",
    problem_type="regression",
    eval_metric="rmse",
    path=model_path_y
).fit(
    train_data=train_data_y,
    time_limit=time_limit,
    presets=presets,
    hyperparameters=hyperparameters,  # 특정 모델만 선택
    verbosity=2
)

# ----------------------
# 12. test 예측
# ----------------------
print("=" * 50)
print("Test 데이터 예측 중...")
print("=" * 50)

pred_x = predictor_x.predict(X_test_feat)
pred_y = predictor_y.predict(X_test_feat)

# 필드 범위로 클립
pred_x = np.clip(pred_x, 0, 105)
pred_y = np.clip(pred_y, 0, 68)

# ----------------------
# 13. submission 생성
# ----------------------
sub = sample_sub.copy()

# X_test에는 game_episode가 있으니, test_index와 align
pred_df = X_test[["game_episode"]].copy()
pred_df["end_x"] = pred_x
pred_df["end_y"] = pred_y

sub = sub.drop(columns=["end_x", "end_y"], errors="ignore")
sub = sub.merge(pred_df, on="game_episode", how="left")

# 제출 파일 이름 생성
submission_filename = f"submission_autogluon_{model_suffix}.csv"
sub.to_csv(submission_filename, index=False)
print(f"Saved {submission_filename}")
print("\n학습 및 예측 완료!")


✓ 핵심 피처 3개 추가 완료 (dist_to_sideline, angle_to_goal_center, time_pos_inter)


/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_3945/2429465320.py:211: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos_in_K)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_3945/2429465320.py:301: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train_feat = X_train_feat.fillna(0)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_3945/2429465320.py:302: FutureWarning: Downcasting objec

LightGBM만 사용 (30분)

X 좌표 모델 학습 시작...

모델 저장 경로: ag_models_x_lgbm_sideline_goalangle_timepos, ag_models_y_lgbm_sideline_goalangle_timepos
사용 모델: LightGBM만
사용 피처: dist_to_sideline, angle_to_goal_center, time_pos_inter


Beginning AutoGluon training ... Time limit = 450s
AutoGluon will save models to "/Users/yangjinmo/Desktop/k_league_ml/ag_models_x_lgbm_sideline_goalangle_timepos/ds_sub_fit/sub_fit_ho"
Train Data Rows:    13720
Train Data Columns: 538
Label Column:       target_x
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    5145.91 MB
	Train Data (Original)  Memory Usage: 63.09 MB (1.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 64 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage


Y 좌표 모델 학습 시작...


	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 64 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 21): ['is_final_team_19', 'is_last_0', 'is_last_1', 'is_last_2', 'is_last_3', 'is_last_4', 'is_last_5', 'is_last_6', 'is_last_7', 'is_last_8', 'is_last_9', 'is_last_10', 'is_last_11', 'is_last_12', 'is_last_13', 'is_last_14', 'is_last_15', 'is_last_16', 'is_last_17', 'is_last_18', 'is_last_19']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all ro

Test 데이터 예측 중...
Saved submission_autogluon_lgbm_sideline_goalangle_timepos.csv

학습 및 예측 완료!


In [ ]:
# ----------------------
# 결과 확인 및 3단계 검증법 (학습 완료 후 실행)
# ----------------------
from autogluon.tabular import TabularPredictor
import pandas as pd
import numpy as np

# ----------------------
# 검증 단계 실행 플래그
# ----------------------
RUN_STEP1_FEATURE_IMPORTANCE = False   # 1단계: Feature Importance 확인
RUN_STEP2_LEADERBOARD = True          # 2단계: 리더보드 점수(RMSE) 확인
RUN_STEP3_SITUATIONAL_ANALYSIS = True # 3단계: 특정 상황에서의 예측값 확인

# 모델 경로
model_suffix = "sideline_goalangle_timepos"
model_path_x = f"ag_models_x_{model_suffix}"
model_path_y = f"ag_models_y_{model_suffix}"
submission_filename = f"submission_autogluon_{model_suffix}.csv"

print(f"모델 경로: {model_path_x}, {model_path_y}")
print(f"제출 파일: {submission_filename}")

# 저장된 모델 로드
try:
    predictor_x = TabularPredictor.load(model_path_x)
    predictor_y = TabularPredictor.load(model_path_y)
    print("✓ 모델 로드 완료")
except Exception as e:
    print(f"⚠ 모델 로드 실패: {e}")
    print("Cell 0을 먼저 실행하여 모델을 학습하세요.")
    raise

print("=" * 70)
print("=" * 70)
print("3단계 검증법: 성능의 실체 확인")
print("=" * 70)
print("=" * 70)

# ============================================================
# 1단계: AutoGluon의 feature_importance 확인
# ============================================================
if RUN_STEP1_FEATURE_IMPORTANCE:
    print("\n" + "=" * 70)
    print("1단계: Feature Importance 확인")
    print("=" * 70)
    print("체크포인트: 새로 만든 피처가 상위 10위 안에 들어오는지 확인")
    print("-" * 70)
    
    try:
        # 새로 추가한 피처 목록 (3개)
        new_features = [
            "dist_to_sideline",      # 사이드라인 압박도
            "angle_to_goal_center",  # 골대 중심과의 물리적 각도
            "time_pos_inter"          # 시간 × 전진 위치 상호작용
        ]
        
        # X 좌표 모델의 feature importance
        print("\n[X 좌표 모델 - Feature Importance (상위 20개)]")
        importance_x = predictor_x.feature_importance(data=train_data_x)
        
        # Series인지 DataFrame인지 확인하여 처리
        if isinstance(importance_x, pd.DataFrame):
            # DataFrame인 경우 첫 번째 컬럼 사용
            importance_x = importance_x.iloc[:, 0]
        
        importance_x_sorted = importance_x.sort_values(ascending=False).head(20)
        print(importance_x_sorted)
        
        print("\n[새로 추가한 피처의 중요도 순위 (X 좌표)]")
        for feat in new_features:
            # 피처 이름이 pivot된 형태일 수 있으므로 패턴 매칭
            matching_features = [f for f in importance_x_sorted.index if feat in str(f)]
            if matching_features:
                for mf in matching_features:
                    rank = list(importance_x_sorted.index).index(mf) + 1
                    importance_val = importance_x_sorted[mf]
                    print(f"  {mf}: 순위 {rank}위, 중요도 {importance_val:.4f}")
            else:
                # 전체에서 찾기
                all_matching = [f for f in importance_x.index if feat in str(f)]
                if all_matching:
                    for mf in all_matching[:3]:  # 상위 3개만 표시
                        rank = list(importance_x.sort_values(ascending=False).index).index(mf) + 1
                        importance_val = importance_x[mf]
                        print(f"  {mf}: 순위 {rank}위, 중요도 {importance_val:.4f}")
                else:
                    print(f"  {feat}: 피처를 찾을 수 없음")
        
        # Y 좌표 모델의 feature importance
        print("\n[Y 좌표 모델 - Feature Importance (상위 20개)]")
        importance_y = predictor_y.feature_importance(data=train_data_y)
        
        # Series인지 DataFrame인지 확인하여 처리
        if isinstance(importance_y, pd.DataFrame):
            # DataFrame인 경우 첫 번째 컬럼 사용
            importance_y = importance_y.iloc[:, 0]
        
        importance_y_sorted = importance_y.sort_values(ascending=False).head(20)
        print(importance_y_sorted)
        
        print("\n[새로 추가한 피처의 중요도 순위 (Y 좌표)]")
        for feat in new_features:
            matching_features = [f for f in importance_y_sorted.index if feat in str(f)]
            if matching_features:
                for mf in matching_features:
                    rank = list(importance_y_sorted.index).index(mf) + 1
                    importance_val = importance_y_sorted[mf]
                    print(f"  {mf}: 순위 {rank}위, 중요도 {importance_val:.4f}")
            else:
                # 전체에서 찾기
                all_matching = [f for f in importance_y.index if feat in str(f)]
                if all_matching:
                    for mf in all_matching[:3]:  # 상위 3개만 표시
                        rank = list(importance_y.sort_values(ascending=False).index).index(mf) + 1
                        importance_val = importance_y[mf]
                        print(f"  {mf}: 순위 {rank}위, 중요도 {importance_val:.4f}")
                else:
                    print(f"  {feat}: 피처를 찾을 수 없음")
            
    except Exception as e:
        import traceback
        print(f"Feature importance 확인 중 오류 발생: {e}")
        print(f"오류 상세: {traceback.format_exc()}")
        print("(모델이 학습되지 않았거나 데이터가 없는 경우 발생할 수 있습니다)")
else:
    print("\n1단계: Feature Importance 확인 건너뜀 (RUN_STEP1_FEATURE_IMPORTANCE = False)")

# ============================================================
# 2단계: 리더보드 점수(RMSE) 비교
# ============================================================
if RUN_STEP2_LEADERBOARD:
    print("\n" + "=" * 70)
    print("2단계: 리더보드 점수(RMSE) 확인")
    print("=" * 70)
    print("체크포인트: X와 Y 좌표의 RMSE가 각각 0.1~0.2 이상 떨어졌는지 확인")
    print("(11.7 -> 11.5 등)")
    print("-" * 70)
    
    # X 좌표 모델 리더보드
    print("\n[X 좌표 모델 - 리더보드 (상위 5개)]")
    leaderboard_x = predictor_x.leaderboard(silent=True)
    print(leaderboard_x.head(5))
    best_rmse_x = abs(leaderboard_x.iloc[0]['score_val'])
    print(f"\n✓ 최고 X 좌표 RMSE: {best_rmse_x:.4f}")
    
    # Y 좌표 모델 리더보드
    print("\n[Y 좌표 모델 - 리더보드 (상위 5개)]")
    leaderboard_y = predictor_y.leaderboard(silent=True)
    print(leaderboard_y.head(5))
    best_rmse_y = abs(leaderboard_y.iloc[0]['score_val'])
    print(f"\n✓ 최고 Y 좌표 RMSE: {best_rmse_y:.4f}")
    
    # Train 데이터로 성능 평가
    try:
        print("\n" + "-" * 70)
        print("[Train 데이터 성능 평가]")
        print("-" * 70)
        
        # X 좌표 평가
        y_pred_train_x = predictor_x.predict(X_train_feat)
        rmse_x = np.sqrt(np.mean((y_train_x - y_pred_train_x) ** 2))
        print(f"X 좌표 RMSE (Train): {rmse_x:.4f}")
        
        # Y 좌표 평가
        y_pred_train_y = predictor_y.predict(X_train_feat)
        rmse_y = np.sqrt(np.mean((y_train_y - y_pred_train_y) ** 2))
        print(f"Y 좌표 RMSE (Train): {rmse_y:.4f}")
        
        print(f"\n✓ 전체 RMSE (Train): {np.sqrt(rmse_x**2 + rmse_y**2):.4f}")
    except NameError:
        print("\n(첫 번째 셀을 먼저 실행해야 Train 데이터 평가가 가능합니다)")
else:
    print("\n2단계: 리더보드 점수 확인 건너뜀 (RUN_STEP2_LEADERBOARD = False)")

# ============================================================
# 3단계: 특정 상황(코너킥/롱볼/후반전)에서의 오차 확인
# ============================================================
if RUN_STEP3_SITUATIONAL_ANALYSIS:
    print("\n" + "=" * 70)
    print("3단계: 특정 상황에서의 예측값 확인")
    print("=" * 70)
    print("체크포인트: 타겟팅한 상황(예: 후반전)에서의 예측값이")
    print("'상식적'으로 변했는지 제출 파일의 통계를 확인")
    print("-" * 70)
    
    try:
        # 제출 파일 로드
        sub = pd.read_csv(submission_filename)
        
        # test 데이터와 merge하여 상황별 분석
        # X_test에는 이미 ep_meta가 merge되어 있음
        if "game_episode" in X_test.columns:
            # 필요한 컬럼만 선택
            needed_cols = ["game_episode"]
            if "game_clock_min" in X_test.columns:
                needed_cols.append("game_clock_min")
            if "period_id" in X_test.columns:
                needed_cols.append("period_id")
            
            test_with_pred = X_test[needed_cols].copy()
            
            # 제출 파일과 merge
            test_with_pred = test_with_pred.merge(
                sub[["game_episode", "end_x", "end_y"]],
                on="game_episode",
                how="left"
            )
        else:
            # X_test가 없으면 제출 파일만 사용
            test_with_pred = sub[["game_episode", "end_x", "end_y"]].copy()
            print("  ⚠ X_test 정보를 찾을 수 없어 전체 통계만 표시합니다.")
        
        print("\n[전체 예측값 통계]")
        print(f"  end_x - mean: {test_with_pred['end_x'].mean():.2f}, "
              f"std: {test_with_pred['end_x'].std():.2f}, "
              f"min: {test_with_pred['end_x'].min():.2f}, "
              f"max: {test_with_pred['end_x'].max():.2f}")
        print(f"  end_y - mean: {test_with_pred['end_y'].mean():.2f}, "
              f"std: {test_with_pred['end_y'].std():.2f}, "
              f"min: {test_with_pred['end_y'].min():.2f}, "
              f"max: {test_with_pred['end_y'].max():.2f}")
        
        # 후반전 분석
        if 'period_id' in test_with_pred.columns:
            period_id_valid = test_with_pred['period_id'].notna().sum()
            if period_id_valid > 0:
                second_half = test_with_pred[test_with_pred['period_id'] == 2]
                first_half = test_with_pred[test_with_pred['period_id'] == 1]
                
                if len(second_half) > 0:
                    print("\n[후반전 예측값 통계]")
                    print(f"  후반전 샘플 수: {len(second_half)} ({len(second_half)/len(test_with_pred)*100:.1f}%)")
                    print(f"  end_x - mean: {second_half['end_x'].mean():.2f}, "
                          f"std: {second_half['end_x'].std():.2f}, "
                          f"min: {second_half['end_x'].min():.2f}, "
                          f"max: {second_half['end_x'].max():.2f}")
                    print(f"  end_y - mean: {second_half['end_y'].mean():.2f}, "
                          f"std: {second_half['end_y'].std():.2f}, "
                          f"min: {second_half['end_y'].min():.2f}, "
                          f"max: {second_half['end_y'].max():.2f}")
                    
                    if len(first_half) > 0:
                        print("\n[전반전 예측값 통계 (비교용)]")
                        print(f"  전반전 샘플 수: {len(first_half)}")
                        print(f"  end_x - mean: {first_half['end_x'].mean():.2f}, "
                              f"std: {first_half['end_x'].std():.2f}")
                        print(f"  end_y - mean: {first_half['end_y'].mean():.2f}, "
                              f"std: {first_half['end_y'].std():.2f}")
                else:
                    print("\n  ⚠ 후반전 데이터가 없습니다.")
            else:
                print("\n  ⚠ period_id 값이 모두 NaN입니다.")
        else:
            print("\n  ⚠ period_id 컬럼이 없어 후반전 분석을 건너뜁니다.")
        
        print("\n[제출 파일 기본 정보]")
        print(f"  제출 파일 행 수: {len(sub)}")
        print(f"\n  제출 파일 샘플:")
        print(sub.head(10))
        print(f"\n  제출 파일 통계:")
        print(sub.describe())
        
    except Exception as e:
        print(f"특정 상황 분석 중 오류 발생: {e}")
        print("(제출 파일이 없거나 데이터 구조가 다른 경우 발생할 수 있습니다)")
else:
    print("\n3단계: 특정 상황 분석 건너뜀 (RUN_STEP3_SITUATIONAL_ANALYSIS = False)")

print("\n" + "=" * 70)
print("3단계 검증 완료!")
print("=" * 70)


모델 경로: ag_models_x_lgbm_sideline_goalangle_timepos, ag_models_y_lgbm_sideline_goalangle_timepos
제출 파일: submission_autogluon_lgbm_sideline_goalangle_timepos.csv
✓ 모델 로드 완료
3단계 검증법: 성능의 실체 확인

1단계: Feature Importance 확인 건너뜀 (RUN_STEP1_FEATURE_IMPORTANCE = False)

2단계: 리더보드 점수(RMSE) 확인
체크포인트: X와 Y 좌표의 RMSE가 각각 0.1~0.2 이상 떨어졌는지 확인
(11.7 -> 11.5 등)
----------------------------------------------------------------------

[X 좌표 모델 - 리더보드 (상위 5개)]
                      model  score_val              eval_metric  \
0       WeightedEnsemble_L3 -11.845297  root_mean_squared_error   
1           LightGBM_BAG_L1 -11.845349  root_mean_squared_error   
2       WeightedEnsemble_L2 -11.845349  root_mean_squared_error   
3           LightGBM_BAG_L2 -11.938057  root_mean_squared_error   
4  WeightedEnsemble_L3_FULL        NaN  root_mean_squared_error   

   pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  \
0       0.368086  24.660502                0.006063           0.007562   
1     